# MCTS特征工程

包含：
1.  加载预清洗的数据集 (`train_augmented.csv`)。
2.  执行特征交互：将LGBM初步分析中最重要的10个特征与特定策略（`selection` 和 `playout`）进行交互，以创建新特征。
3.  进行特征筛选实验：使用LightGBM和递归特征消除（RFE）来识别对模型影响最大的特征。
4.  准备并导出最终数据集：生成一个仅包含所选特征的CSV文件，供建模与优化团队使用。

此Notebook的输出将是 `train_final_features.csv` 文件，作为后续工作流程的输入。

## 1. 环境设置与数据加载

In [16]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [17]:
DATA_PATH = 'train_augmented.csv'
df = None

print(f"正在查找文件 '{DATA_PATH}' ...")

if not os.path.exists(DATA_PATH):
    print(f"!!! 错误：在Colab文件系统中未找到 '{DATA_PATH}'。")
    print("--- 请现在上传文件: ---")
    from google.colab import files
    uploaded = files.upload()

    if DATA_PATH not in uploaded:
        raise ValueError(f"上传失败或上传的文件名不是 '{DATA_PATH}'。请检查文件名并重试。")
    print(f"\n文件 '{DATA_PATH}' 上传成功！")
else:
    print(f"成功找到文件 '{DATA_PATH}'。")

print("\n正在加载数据...")
try:
    df = pd.read_csv(DATA_PATH)
    print("\n数据集加载成功！")
    print("以下是数据的前5行和基本信息：")
    display(df.head())
    print("\n数据集信息:")
    df.info()
except Exception as e:
    print(f"\n加载文件时发生错误: {e}")
    df = None

正在查找文件 'train_augmented.csv' ...
成功找到文件 'train_augmented.csv'。

正在加载数据...

数据集加载成功！
以下是数据的前5行和基本信息：


,Stochastic,Asymmetric,AsymmetricForces,AsymmetricPiecesType,PlayersWithDirections,Cooperation,Team,Shape,SquareShape,HexShape,TriangleShape,DiamondShape,RectangleShape,SpiralShape,CircleShape,StarShape,SquarePyramidalShape,RegularShape,PolygonShape,Tiling,SquareTiling,HexTiling,TriangleTiling,SemiRegularTiling,MorrisTiling,CircleTiling,ConcentricTiling,SpiralTiling,AlquerqueTiling,MancalaBoard,MancalaStores,MancalaTwoRows,MancalaThreeRows,MancalaFourRows,MancalaSixRows,MancalaCircular,AlquerqueBoard,AlquerqueBoardWithOneTriangle,AlquerqueBoardWithTwoTriangles,AlquerqueBoardWithFourTriangles,AlquerqueBoardWithEightTriangles,ThreeMensMorrisBoard,ThreeMensMorrisBoardWithTwoTriangles,NineMensMorrisBoard,StarBoard,CrossBoard,KintsBoard,PachisiBoard,FortyStonesWithFourGapsBoard,Track,...,FairyChessComponent,PloyComponent,ShogiComponent,XiangqiComponent,StrategoComponent,JanggiComponent,CheckersComponent,BallComponent,TaflComponent,DiscComponent,MarkerComponent,StackType,Stack,Symbols,ShowPieceValue,ShowPieceState,Implementation,State,StateType,StackState,PieceState,SiteState,SetSiteState,VisitedSites,Variable,SetVar,RememberValues,ForgetValues,SetPending,InternalCounter,SetInternalCounter,PlayerValue,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,utility_agent1,agent1_selection,agent1_exploration,agent1_playout,agent1_score_bounds,agent2_selection,agent2_exploration,agent2_playout,agent2_score_bounds,is_augmented
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,298.07,18877.17,-0.466667,ProgressiveHistory,0.1,MAST,0.0,ProgressiveHistory,0.600000,Random200,0.0,0.0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,298.07,18877.17,-0.333333,ProgressiveHistory,0.1,MAST,0.0,UCB1GRAVE,0.600000,NST,1.0,0.0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,298.07,18877.17,-0.066667,ProgressiveHistory,0.1,MAST,1.0,UCB1,0.100000,NST,0.0,0.0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,298.07,18877.17,-0.333333,ProgressiveHistory,0.1,MAST,1.0,UCB1,0.600000,NST,0.0,0.0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,298.07,18877.17,-0.333333,ProgressiveHistory,0.1,MAST,1.0,UCB1GRAVE,1.414214,NST,0.0,0.0



数据集信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131958 entries, 0 to 131957
Columns: 616 entries, Stochastic to is_augmented
dtypes: float64(466), int64(142), object(8)
memory usage: 620.2+ MB


### 初始数据准备

处理分类特征并定义我们的目标和特征集

In [18]:
TARGET = 'utility_agent1'

nan_count = df[TARGET].isnull().sum()
if nan_count > 0:
    print(f"发现目标变量 '{TARGET}' 中有 {nan_count} 个NaN值。正在移除这些行...")
    df = df.dropna(subset=[TARGET])
    print(f"移除后，数据集的新维度为: {df.shape}")

# 建模组需要此列进行分组交叉验证
group_col_options = ['GameRulesetName', 'game_ruleset_name']
GROUP = None
for col in group_col_options:
    if col in df.columns:
        GROUP = col
        break


# 将所有 object 类型的列转换为 'category' 类型，以便LightGBM处理
for col in df.columns:
    if df[col].dtype == 'object':
        # 确保不转换已找到的分组列，如果它恰好是object类型
        if col != GROUP:
            df[col] = df[col].astype('category')

print('已将 object 类型的列转换为 category 类型。')

发现目标变量 'utility_agent1' 中有 2 个NaN值。正在移除这些行...
移除后，数据集的新维度为: (131956, 616)
已将 object 类型的列转换为 category 类型。


## 2. 特征交互

根据EDA的发现，初始LightGBM模型中最重要的10个特征是：
1.  `agent2_selection`
2.  `agent1_selection`
3.  `AdvantageP1`
4.  `agent1_playout`
5.  `agent2_playout`
6.  `agent2_exploration`
7.  `PlayoutsPerSecond`
8.  `DurationTurnsNotTimeouts`
9.  `GameTreeComplexity`
10. `agent1_exploration`

我们现在将通过结合这些顶级特征与 `selection` 和 `playout` 策略来创建交互特征。这些策略是分类的，因此我们首先对它们进行标签编码，以创建有意义的数值交互。

In [19]:
top_10_features = [
    'agent2_selection', 'agent1_selection', 'AdvantageP1', 'agent1_playout',
    'agent2_playout', 'agent2_exploration', 'PlayoutsPerSecond',
    'DurationTurnsNotTimeouts', 'GameTreeComplexity', 'agent1_exploration'
]

agent_strategy_features = ['agent1_selection', 'agent1_playout', 'agent2_selection', 'agent2_playout']

# 对agent策略特征进行标签编码，以便为交互创建数值表示
for col in agent_strategy_features:
    le = LabelEncoder()
    df[f'{col}_encoded'] = le.fit_transform(df[col])

print("已为智能体策略特征创建标签编码版本。")

# 创建交互特征
interaction_features_to_create = [f for f in top_10_features if f not in agent_strategy_features and df[f].dtype != 'category']

for top_feat in interaction_features_to_create:
    # agent1交互
    df[f'int_{top_feat}_x_a1_select'] = df[top_feat] * df['agent1_selection_encoded']
    df[f'int_{top_feat}_x_a1_playout'] = df[top_feat] * df['agent1_playout_encoded']

    # agent2交互
    df[f'int_{top_feat}_x_a2_select'] = df[top_feat] * df['agent2_selection_encoded']
    df[f'int_{top_feat}_x_a2_playout'] = df[top_feat] * df['agent2_playout_encoded']

print(f"创建了 {len(interaction_features_to_create) * 4} 个新的交互特征。")
print('新数据集维度:', df.shape)

已为智能体策略特征创建标签编码版本。
创建了 24 个新的交互特征。
新数据集维度: (131956, 644)


## 3. 特征筛选实验

随着新交互特征的加入，我们的特征空间变大了。为了避免过拟合，并为建模团队降低模型复杂性，我们将进行特征筛选实验。

我们将使用带有LightGBM模型的 **递归特征消除（Recursive Feature Elimination, RFE）**。RFE通过迭代训练模型并移除最不重要的特征，直到达到所需的特征数量。这有助于我们从数据中提炼出最强大和最具预测性的信号。

我们的目标是筛选出**前100个特征**。

In [20]:
from sklearn.feature_selection import RFE

# 准备用于特征筛选的数据
y = df[TARGET]

# 从特征矩阵X中移除目标列和分组列
columns_to_drop = [TARGET]
if GROUP and GROUP in df.columns:
    columns_to_drop.append(GROUP)
X = df.drop(columns=columns_to_drop)

# 编码后的策略特征是辅助性的，现在可以删除它们
encoded_helpers = [f'{col}_encoded' for col in agent_strategy_features]
X = X.drop(columns=encoded_helpers, errors='ignore').copy()

# 这是为了解决 RFE 无法处理 'category' 类型的列的问题
for col in X.columns:
    if X[col].dtype.name == 'category':
        print(f"正在将分类特征 '{col}' 转换为数字编码...")
        X[col] = X[col].cat.codes

# 确保所有特征名称都是字符串，以供LightGBM使用
X.columns = [str(col) for col in X.columns]

# 初始化用于RFE的LightGBM回归器
lgbm = lgb.LGBMRegressor(
    random_state=42,
    n_estimators=100, # 用于特征重要性计算的合理数量
    n_jobs=-1
)

# 初始化RFE
rfe = RFE(
    estimator=lgbm,
    n_features_to_select=100,
    step=0.1, # 每次迭代移除10%的特征
    verbose=1
)

# 拟合RFE
print("开始递归特征消除... 这可能需要几分钟时间。")
rfe.fit(X, y)
print("RFE拟合完成。")

# 获取筛选出的特征
selected_features = X.columns[rfe.support_].tolist()
print(f"\n筛选出 {len(selected_features)} 个特征:")
print(selected_features)

正在将分类特征 'CountPiecesNextComparison' 转换为数字编码...
正在将分类特征 'Directions' 转换为数字编码...
正在将分类特征 'AllDirections' 转换为数字编码...
正在将分类特征 'OrthogonalDirection' 转换为数字编码...
正在将分类特征 'agent1_selection' 转换为数字编码...
正在将分类特征 'agent1_playout' 转换为数字编码...
正在将分类特征 'agent2_selection' 转换为数字编码...
正在将分类特征 'agent2_playout' 转换为数字编码...
开始递归特征消除... 这可能需要几分钟时间。
Fitting estimator with 639 features.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.258593 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19122
[LightGBM] [Info] Number of data points in the train set: 131956, number of used features: 584
[LightGBM] [Info] Start training from score 0.038716
Fitting estimator with 576 features.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.243305 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough,

## 4. 准备并导出最终数据集

经过特征交互和递归特征消除（RFE）实验，我们从包含全部原始特征和新建交互特征总计超过600个候选特征筛选出了最具预测能力的100个特征。
这份数据集包含了以下关键部分：
1. 最佳特征组合：由RFE筛选出的100个最具影响力的特征。这是一个综合了原始特征和新建交互特征的优化组合。
2.  目标变量 (`utility_agent1`)：供模型进行监督学习。
3.  分组变量 (`GameRulesetName`)：我们已将其完好保留，供建模组进行严格的分组交叉验证（GroupKFold）

In [21]:
final_cols = selected_features + [TARGET]
if GROUP and GROUP in df.columns:
    final_cols.append(GROUP)

df_final = df[final_cols]

print("最终数据集维度:", df_final.shape)
df_final.head()

最终数据集维度: (131956, 101)


,Asymmetric,NumPlayableSitesOnBoard,NumRows,NumCorners,NumDirections,NumOrthogonalDirections,NumOuterSites,NumEdges,NumTopSites,NumRightSites,NumCentreSites,NumComponentsTypePerPlayer,NumStartComponentsHand,NumStartComponents,PassDecisionFrequency,StepDecisionFrequency,StepDecisionToEmptyFrequency,SlideDecisionToEnemyFrequency,HopDecisionEnemyToEmptyFrequency,FromToDecisionBetweenContainersFrequency,SowFrequency,SetNextPlayerFrequency,HopCaptureFrequency,CaptureSequenceFrequency,Line,EliminatePiecesEndFrequency,ReachEndFrequency,ReachWinFrequency,NoMovesDraw,DrawFrequency,DurationActions,DurationMoves,DurationTurns,DurationTurnsStdDev,DurationTurnsNotTimeouts,DecisionMoves,GameTreeComplexity,StateTreeComplexity,BoardCoverageFull,BoardCoverageUsed,AdvantageP1,Balance,Completion,OutcomeUniformity,BoardSitesOccupiedAverage,BoardSitesOccupiedMaximum,BoardSitesOccupiedVariance,BoardSitesOccupiedChangeAverage,BoardSitesOccupiedChangeSign,BoardSitesOccupiedMaxIncrease,...,BranchingFactorAverage,BranchingFactorMedian,BranchingFactorMaximum,BranchingFactorVariance,BranchingFactorChangeAverage,BranchingFactorChangeSign,BranchingFactorChangeLineBestFit,BranchingFactorChangeMaxIncrease,BranchingFactorChangeMaxDecrease,DecisionFactorAverage,DecisionFactorMedian,DecisionFactorVariance,DecisionFactorChangeAverage,DecisionFactorChangeSign,DecisionFactorMaxIncrease,MoveDistanceAverage,PieceNumberAverage,PieceNumberMaximum,PieceNumberVariance,PieceNumberChangeAverage,PieceNumberMaxDecrease,ScoreDifferenceMaxIncrease,GreaterThanOrEqual,PlayoutsPerSecond,MovesPerSecond,agent1_selection,agent1_exploration,agent1_playout,agent2_selection,agent2_exploration,agent2_playout,int_AdvantageP1_x_a1_select,int_AdvantageP1_x_a1_playout,int_AdvantageP1_x_a2_select,int_AdvantageP1_x_a2_playout,int_agent2_exploration_x_a2_select,int_PlayoutsPerSecond_x_a1_select,int_PlayoutsPerSecond_x_a1_playout,int_PlayoutsPerSecond_x_a2_select,int_PlayoutsPerSecond_x_a2_playout,int_DurationTurnsNotTimeouts_x_a1_select,int_DurationTurnsNotTimeouts_x_a1_playout,int_DurationTurnsNotTimeouts_x_a2_select,int_DurationTurnsNotTimeouts_x_a2_playout,int_GameTreeComplexity_x_a1_select,int_GameTreeComplexity_x_a1_playout,int_GameTreeComplexity_x_a2_select,int_GameTreeComplexity_x_a2_playout,int_agent1_exploration_x_a1_select,utility_agent1
0,0,48,8,6.0,5.0,5.0,21.0,120,4,1,3,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.63,0.0,0.0,0,0.0,0.0,0.0,0.0,0.36,175.14,63.24,23.45,2.02,23.45,0.99,81.13,28.9,0.19,0.94,0.5,0.88,1.0,-0.03,0.5,0.94,0.07,0.01,0.72,0.02,...,19.4,17.12,48.0,230.17,-0.73,-0.49,-0.61,37.6,-38.58,19.63,17.56,229.16,-0.73,-0.49,37.6,0.0,23.86,45.2,169.31,0.72,0.0,0.0,0.0,298.07,18877.17,ProgressiveHistory,0.1,MAST,ProgressiveHistory,0.600000,Random200,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.00,596.14,0.0,0.0,0.00,46.90,0.0,0.0,0.00,162.26,0.0,-0.466667
1,0,48,8,6.0,5.0,5.0,21.0,120,4,1,3,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.63,0.0,0.0,0,0.0,0.0,0.0,0.0,0.36,175.14,63.24,23.45,2.02,23.45,0.99,81.13,28.9,0.19,0.94,0.5,0.88,1.0,-0.03,0.5,0.94,0.07,0.01,0.72,0.02,...,19.4,17.12,48.0,230.17,-0.73,-0.49,-0.61,37.6,-38.58,19.63,17.56,229.16,-0.73,-0.49,37.6,0.0,23.86,45.2,169.31,0.72,0.0,0.0,0.0,298.07,18877.17,ProgressiveHistory,0.1,MAST,UCB1GRAVE,0.600000,NST,0.0,0.0,1.0,0.5,1.200000,0.0,0.0,596.14,298.07,0.0,0.0,46.90,23.45,0.0,0.0,162.26,81.13,0.0,-0.333333
2,0,48,8,6.0,5.0,5.0,21.0,120,4,1,3,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.63,0.0,0.0,0,0.0,0.0,0.0,0.0,0.36,175.14,63.24,23.45,2.02,23.45,0.99,81.13,28.9,0.19,0.94,0.5,0.88,1.0,-0.03,0.5,0.94,0.07,0.01,0.72,0.02,...,19.4,17.12,48.0,230.17,-0.73,-0.49,-0.61,37.6,-38.58,19.63,17.56,229.16,-0.73,-0.49,37.6,0.0,23.86,45.2,169.31,0.72,0.0,0.0,0.0,298.07,18877.17,ProgressiveHistory,0.1,MAST,UCB1,0.100000,NST,0.0,0.0,0.5,0.5,0.100000,0.0,0.0,298.07,298.07,0.0,0.0,23.45,23.45,0.0,0.0,81.13,81.13,0.0,-0.066667
3,0,48,8,6.0,5.0,5.0,21.0,120,4,1,3,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.63,0.0,0.0,0,0.0,0.0,0.0,0.0,0.36,175.1

In [22]:
from google.colab import files

output_filename = 'train_final_features.csv'
df_final.to_csv(output_filename, index=False)

print(f"最终数据集已保存为 {output_filename}。")
files.download(output_filename)

最终数据集已保存为 train_final_features.csv。


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 给建模与优化组的建议

我们给到的`train_final_features.csv` 文件会是你们建模时候所需要的清洗好的数据集。

**给下一组同学的建议：**
1.  **加载 `train_final_features.csv`**。所有必要的特征、目标变量和分组变量都已包含在内。
2.  **处理分类特征**：所有 `object` 类型的列都已转换为 `category` 类型，非常适合直接在LightGBM和CatBoost中使用。
3.  **交叉验证**：使用 `sklearn.model_selection.GroupKFold` 并将 `GameRulesetName` 列作为 `groups` 参数来构建你们的验证框架。
4.  **建模**：按计划进行LightGBM和CatBoost模型的构建、调优（使用Optuna）和集成。